In [3]:
import explicit_statetespace as efss
import ssr_kronecker_vector as ssr
import Utilityfunctions as utils
import kronecker_vector as fss
import numpy as np

In [10]:
n = 3
sparsity = 0.5
theta = utils.random_theta(n, sparsity)
print(theta)

[[ 0.9703618   0.         -0.5744573   0.        ]
 [ 1.40758012  0.17806066  1.69484597  0.        ]
 [ 0.          0.          1.22961169  0.78783247]
 [-0.48808697  0.          0.         -0.3271231 ]]


In [17]:
# Test explicit matrix vector product vs shuffle trick kronecker vector product
p = np.zeros(2**(2*n+1))
p[0] = 1
assert(np.allclose(
    efss.build_q(theta)@p,
    fss.qvec(theta, p, diag=True)))

In [6]:
state = np.random.randint(2, size=2*n+1)
length = 2**sum(state)

In [13]:
# Test explicit ssr vs. shuffle trick ssr
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    assert(np.allclose(
        efss.ssr_build_q(state, np.exp(theta)) @ p,
        ssr.kronvec(log_theta=theta, p=p, n=n, state=state)
    ))


In [5]:
mut = [state[j] + 2 * state[j + 1] for j in range(0, 2 * n, 2)]
mut.append(state[-1])
mut = np.array(mut)


In [6]:
# Test explicit ssr vs. shuffle trick ssr in sync
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    for i in range(n):
        assert(np.allclose(
            efss.sync_ssr_q(mut=mut, theta=np.exp(theta), i=i, n=n) @ p,
            ssr.kronvec_sync(log_theta=theta, p=p, n=n, i=i, state=state)
        ))


In [7]:
# Test explicit ssr vs. shuffle trick ssr in async prim
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    for i in range(n):
        assert(np.allclose(
            efss.prim_ssr_q(mut=mut, theta=np.exp(theta), i=i, n=n) @ p,
            ssr.kronvec_prim(log_theta=theta, p=p, n=n, i=i, state=state)
        ))


In [8]:
# Test explicit ssr vs. shuffle trick ssr in async met
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    for i in range(n):
        assert(np.allclose(
            efss.met_ssr_q(mut=mut, theta=np.exp(theta), i=i, n=n) @ p,
            ssr.kronvec_met(log_theta=theta, p=p, n=n, i=i, state=state)
        ))


In [9]:
# Test explicit ssr vs. shuffle trick ssr in seeding
for j in range(length):
    p = np.zeros(length)
    p[j] = 1
    assert(np.allclose(
        efss.seeding_ssr_q(mut=mut, theta=np.exp(theta), n=n) @ p,
        ssr.kronvec_seed(log_theta=theta, p=p, n=n, state=state)
    ))


In [10]:
# Test no diag
for j in range(length):
    p = np.zeros(2**sum(state))
    p[j] = 1
    q = ssr.kronvec(log_theta=theta, p=p, n=n, state=state)
    q[j] = 0
    assert(np.allclose(
        q,
        ssr.kronvec(log_theta=theta, p=p, n=n, state=state, diag=False)
    ))
